In [215]:
import numpy as np
import pandas as pd
import requests
import json
import folium
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import time
import math
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, GeoJSONDataSource
from bokeh.tile_providers import ESRI_IMAGERY, get_provider
import geopandas as gpd
from pyproj import Transformer, CRS
from shapely.ops import transform

In [14]:
with open('variables/city_centers.json') as f_in:
    city_centers = json.load(f_in)

sf_lat = city_centers['sf'][0]
sf_lon = city_centers['sf'][1]
chicago_lat = city_centers['chicago'][0]
chicago_lon = city_centers['chicago'][1]
nyc_lat = city_centers['nyc'][0]
nyc_lon = city_centers['nyc'][1]

In [2]:
with open('json_data/sf_venues_results.json') as f_in: 
    sf_venues_results = json.load(f_in)
with open('json_data/chicago_venues_results.json') as f_in:
    chicago_venues_results = json.load(f_in)
with open('json_data/nyc_venues_results.json') as f_in:
    nyc_venues_results = json.load(f_in)

In [176]:
california = gpd.read_file('shape_data/california/california.shp')
illinois = gpd.read_file('shape_data/illinois/illinois.shp')
new_york = gpd.read_file('shape_data/new_york/new_york.shp')

In [177]:
sf_shp = california[california['NAME'] == 'San Francisco']
chicago_shp = illinois[illinois['NAME'] == 'Chicago']
nyc_shp = new_york[new_york['NAME'] == 'New York']

In [51]:
unique_ids = set()

sf_data = {
    'latitudes': [],
    'longitudes': [],
    'x_coords': [],
    'y_coords': [],
    'labels': []
}

for i, venue in enumerate(sf_venues_results):
    items_list = venue['response']['groups'][0]['items']
    for item in items_list:
        item_id = item['venue']['id']
        if item_id in unique_ids:
            continue
        else:
            unique_ids.add(item_id)
            sf_data['latitudes'].append(item['venue']['location']['lat'])
            sf_data['longitudes'].append(item['venue']['location']['lng'])
            sf_data['labels'].append(item['venue']['categories'][0]['name'])

for lat, lon in zip(sf_data['latitudes'], sf_data['longitudes']):
    x, y = latlon2mercator(lat, lon)
    sf_data['x_coords'].append(x)
    sf_data['y_coords'].append(y)

In [40]:
unique_ids = set()

chicago_data = {
    'latitudes': [],
    'longitudes': [],
    'x_coords': [],
    'y_coords': [],
    'labels': []
}

for i, venue in enumerate(chicago_venues_results):
    items_list = venue['response']['groups'][0]['items']
    for item in items_list:
        item_id = item['venue']['id']
        if item_id in unique_ids:
            continue
        else:
            unique_ids.add(item_id)
            chicago_data['latitudes'].append(item['venue']['location']['lat'])
            chicago_data['longitudes'].append(item['venue']['location']['lng'])
            chicago_data['labels'].append(item['venue']['categories'][0]['name'])

for lat, lon in zip(chicago_data['latitudes'], chicago_data['longitudes']):
    x, y = latlon2mercator(lat, lon)
    chicago_data['x_coords'].append(x)
    chicago_data['y_coords'].append(y)

In [194]:
unique_ids = set()

nyc_data = {
    'latitudes': [],
    'longitudes': [],
    'x_coords': [],
    'y_coords': [],
    'labels': []
}

for i, venue in enumerate(nyc_venues_results):
    items_list = venue['response']['groups'][0]['items']
    for item in items_list:
        item_id = item['venue']['id']
        if item_id in unique_ids:
            continue
        else:
            unique_ids.add(item_id)
            nyc_data['latitudes'].append(item['venue']['location']['lat'])
            nyc_data['longitudes'].append(item['venue']['location']['lng'])
            nyc_data['labels'].append(item['venue']['categories'][0]['name'])

for lat, lon in zip(nyc_data['latitudes'], nyc_data['longitudes']):
    x, y = latlon2mercator(lat, lon)
    nyc_data['x_coords'].append(x)
    nyc_data['y_coords'].append(y)

In [160]:
samp_x, samp_y = latlon2mercator(37.76, -122.44)

In [173]:
tile_provider = get_provider(OSM)
sf_x, sf_y = latlon2mercator(sf_lat, sf_lon)
# p = figure(x_range = (sf_x - 100, sf_x + 100), y_range= (sf_y - 100, sf_y + 100),
#            x_axis_type = "mercator", y_axis_type = "mercator")
p = figure(x_axis_type="mercator", y_axis_type="mercator")
p.circle(x = 'x_coords', y = 'y_coords', source = sf_data)
p.annulus(x = samp_x, y = samp_y, inner_radius = 0, outer_radius = 2030.0, 
          fill_color = 'green', fill_alpha = 0.5, line_alpha = 0)
p.annulus(x = samp_x + 800, y = samp_y, inner_radius = 0, outer_radius = 2030.0, 
          fill_color = 'green', fill_alpha = 0.5, line_alpha = 0)
p.annulus(x = samp_x, y = samp_y + 800, inner_radius = 0, outer_radius = 2030.0, 
          fill_color = 'green', fill_alpha = 0.5, line_alpha = 0)
p.annulus(x = samp_x + 800, y = samp_y + 800, inner_radius = 0, outer_radius = 2030.0, 
          fill_color = 'green', fill_alpha = 0.5, line_alpha = 0)
p.add_tile(tile_provider)
show(p)

In [25]:
tile_provider = get_provider(ESRI_IMAGERY)

# range bounds supplied in web mercator coordinates
p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)

show(p)

In [50]:
def latlon2mercator(lat, lon):
    r_earth = 6378137.000
    x = r_earth * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

In [52]:
def kil2mil(km):
    return km * 0.621371

In [53]:
# Returns (approximate) distance in kilometers or miles.
def calc_dist_latlon(lat_1, lon_1, lat_2, lon_2, unit = 'km'):
    r_earth = 6371 # avg radius of Earth in km
    d_lat = np.radians(lat_2 - lat_1)
    d_lon = np.radians(lon_2 - lon_1)
    
#     a = ((math.sin(d_lat/2))**2 + 
#          (math.sin(d_lon/2))**2 + 
#          (math.cos(deg2rad(lat_1)) * math.cos(deg2rad(lat_2))))
    
#     print(math.cos(deg2rad(lat_1)))
#     print(math.cos(deg2rad(lat_2)))
#     print(a)

    a = ((math.sin(d_lat/2) * math.sin(d_lat/2)) + 
         (math.cos(np.radians(lat_1)) * math.cos(np.radians(lat_2)) * 
         math.sin(d_lon/2) * math.sin(d_lon/2)))
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = r_earth * c
    
    if unit == 'km':
        return d
    elif unit == 'mi':
        return kil2mil(d)
    else:
        print("Units not recognized.")
        return 0

In [54]:
print(chicago_data['latitudes'][0])
print(chicago_data['longitudes'][0])

41.658275
-87.608822


In [83]:
lat_1 = 41.66000
lon_1 = -87.61
lat_2 = 41.67446
lon_2 = -87.61

In [84]:
calc_dist_latlon(lat_1, lon_1, lat_2, lon_2, 'mi')

0.9990891579687212

In [87]:
print(latlon2mercator(lat_1, lon_1)[1] - latlon2mercator(lat_2, lon_2)[1])

-2154.8057680875063


In [112]:
lat_1 = 41.66
lon_1 = -87.62936
lat_2 = 41.66
lon_2 = -87.61000

In [113]:
calc_dist_latlon(lat_1, lon_1, lat_2, lon_2, 'mi')

0.9993588150024879

In [114]:
print(latlon2mercator(lat_1, lon_1)[0] - latlon2mercator(lat_2, lon_2)[0])

-2155.1453417595476


In [115]:
print(sf_data['latitudes'][0])
print(sf_data['longitudes'][0])

37.71047
-122.46752


In [120]:
lat_1 = 37.71
lon_1 = -122.47
lat_2 = 37.72446
lon_2 = -122.47

In [121]:
calc_dist_latlon(lat_1, lon_1, lat_2, lon_2, 'mi')

0.9990891579682302

In [122]:
print(latlon2mercator(lat_1, lon_1)[1] - latlon2mercator(lat_2, lon_2)[1])

-2034.8916148012504


In [154]:
lat_1 = 37.71
lon_1 = -122.48830
lat_2 = 37.71
lon_2 = -122.47000

In [155]:
calc_dist_latlon(lat_1, lon_1, lat_2, lon_2, 'mi')

1.0002939531870083

In [156]:
print(latlon2mercator(lat_1, lon_1)[0] - latlon2mercator(lat_2, lon_2)[0])

-2037.1466815192252


In [245]:
# project = Transformer.from_proj(
#     pyproj.Proj(init='epsg:4326'), # source coordinate system
#     pyproj.Proj(init='epsg:3785')) # destination coordinate system

crs_source = CRS("EPSG:4326")
crs_dest = CRS("EPSG:3785")

transformer = Transformer.from_crs(crs_source, crs_dest)

In [247]:
sf_polygon = sf_shp.iloc[0]['geometry']
single_polygon = sf_polygon[3]
len(single_polygon.interiors)
for x in single_polygon.exterior.coords:
    print(transformer.transform(x[1], x[0]))
    break

(-13638249.862361172, 4548510.400085907)


In [ ]:
sf_geosource = GeoJSONDataSource(geojson = sf_shp.to_json())
chicago_geosource = GeoJSONDataSource(geojson = chicago_shp.to_json())
nyc_geosource = GeoJSONDataSource(geojson = nyc_shp.to_json())

In [248]:
p = figure(x_axis_type = "mercator", y_axis_type = "mercator")
# p.patches('xs', 'ys', source = sf_geosource, alpha = 0.5, color = 'green')
p.circle('x_coords', 'y_coords', source = sf_data)
p.circle(-13638249.862361172, 4548510.400085907, color = 'red')

tile_provider = get_provider(OSM)
p.add_tile(tile_provider)

show(p)

In [202]:
p = figure(x_axis_type = "mercator", y_axis_type = "mercator")
p.patches('xs', 'ys', source = chicago_geosource, alpha = 0.5, color = 'green')
p.circle('x_coords', 'y_coords', source = chicago_data)

tile_provider = get_provider(OSM)
p.add_tile(tile_provider)

show(p)

In [195]:
p = figure(x_axis_type = "mercator", y_axis_type = "mercator")
p.patches('xs', 'ys', source = nyc_geosource, alpha = 0.5, color = 'green')
p.circle('longitudes', 'latitudes', source = nyc_data)

tile_provider = get_provider(OSM)
p.add_tile(tile_provider)

show(p)